### [758\. Bold Words in String](https://leetcode.com/problems/bold-words-in-string/)

Difficulty: **Easy**


Given a set of keywords `words` and a string `S`, make all appearances of all keywords in `S` bold. Any letters between `<b>` and `</b>` tags become bold.

The returned string should use the least number of tags possible, and of course the tags should form a valid combination.

For example, given that `words = ["ab", "bc"]` and `S = "aabcd"`, we should return `"a<b>abc</b>d"`. Note that returning `"a<b>a<b>b</b>c</b>d"` would use more tags, so it is incorrect.

**Note:**

1.  `words` has length in range `[0, 50]`.
2.  `words[i]` has length in range `[1, 10]`.
3.  `S` has length in range `[0, 500]`.
4.  All characters in `words[i]` and `S` are lowercase letters.

Note: Same as #616. Add Bold Tag in String

In [17]:
# trie + mask array + groupby
from typing import List
import itertools

class Solution:
    def boldWords(self, words: List[str], S: str) -> str:
        # construct trie
        root = {} # every node is a dict in which key is the pointer to next node, value is next node's children
        for w in words:
            cur_node = root
            for ch in w:
                cur_node = cur_node.setdefault(ch, {}) # return cur_node[ch] if ch exists in cur_node
            cur_node['#'] = 0 # mark ending as #
        
        L = len(S)
        masks = [False] * L
        for i in range(L):
            cur = root
            k = i
            for j in range(i, L):
                if S[j] not in cur:
                    break # not match
                cur = cur[S[j]]
                if '#' in cur:
                    k = j+1
            masks[i:k] = [True] * (k-i)
            #for l in range(i, k):
            #    masks[l] = True
        
        res = ''
        for key, group in itertools.groupby(zip(S, masks), lambda t: t[1]):
            if key:
                res += '<b>'
            res += "".join(t[0] for t in group)
            if key:
                res += '</b>'
        return res

In [25]:
# trie + merge overlapping intervals
# https://leetcode.com/problems/bold-words-in-string/discuss/247076/Optimized-Python-Solution-using-Trie-Tree-and-Merge-Intervals
from typing import List

class Solution:
    def boldWords(self, words: List[str], S: str) -> str:
        # construct trie
        root = {} # every node is a dict in which key is the pointer to next node, value is next node's children
        for w in words:
            cur_node = root
            for ch in w:
                cur_node = cur_node.setdefault(ch, {}) # return cur_node[ch] if ch exists in cur_node
            cur_node['#'] = 0 # mark ending as #
        
        intervals = []
        def addInterval(interval):
            nonlocal intervals
            if intervals and intervals[-1][1] >= interval[0]:
                if intervals[-1][1] < interval[1]: # overlapping two intervals found
                    intervals[-1][1] = interval[1]
            else:
                intervals += interval,
                
        L = len(S)
        for i in range(L):
            cur = root
            k = i
            for j in range(i, L):
                if S[j] not in cur:
                    break # not match
                cur = cur[S[j]]
                if '#' in cur:
                    k = j+1
            if k != i:
                addInterval([i, k])
        
        res = ""
        prev_end = 0
        for start, end in intervals:
            res += S[prev_end:start] + '<b>' + S[start:end] + "</b>"
            prev_end = end
        return res + S[prev_end:]

In [26]:
Solution().boldWords(["ab","bc"], "aabcd")

'a<b>abc</b>d'

In [27]:
Solution().boldWords(["e","ad","ce","a","b"], "adcaddeced")

'<b>ad</b>c<b>ad</b>d<b>ece</b>d'